In [1]:
import ctypes
from pynq import Overlay
from pynq import allocate
import numpy as np
import random
import pynq

from pynq.pmbus import DataRecorder
from pynq import Device

from framework.image import SobelGradient
from framework.losses import MutualInformationLossNative, MutualInformationLossFPGA
from framework.optimizers import GradientDescentOptimizer
from framework.transforms import RotateShiftTransform

import pydicom
import cv2
from matplotlib import pyplot as plt
from tqdm import tqdm
from timeit import default_timer as timer
import os
import time

In [2]:
_lib = ctypes.CDLL("framework/losses.lib")
_lib.parzen_mutual_information_matrix.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags='C_CONTIGUOUS')
]

_lib.parzen_mutual_information_grad.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags='C_CONTIGUOUS')
]

_lib.parzen_mutual_information_point.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    np.ctypeslib.ndpointer(dtype=np.uint8, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_int,
    np.ctypeslib.ndpointer(dtype=np.float32, ndim=1, flags='C_CONTIGUOUS')
]

overlay = Overlay('./build/assets/mutual_information_gradient_matrix/gem_wrapper.bit')
mi_ip = overlay.mutual_information_derived_master_0

size = 512
image_dim = size*size
matrix_dim = 256*256
image_1 = allocate(shape=(image_dim,), dtype=np.uint8)
image_2 = allocate(shape=(image_dim,), dtype=np.uint8)
res = allocate(shape=(matrix_dim,), dtype=np.float32)

res_cpu = np.empty(matrix_dim, dtype=np.float32)

In [3]:
dcm1 = pydicom.dcmread('IM10_a.dcm')
img1 = cv2.resize(dcm1.pixel_array, dsize=(size,size))
img1 = cv2.normalize(img1, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
img1 = img1.flatten()

dcm2 = pydicom.dcmread('IM10_b.dcm')
img2 = cv2.resize(dcm2.pixel_array, dsize=(size,size))
img2 = cv2.normalize(img2, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
img2 = img2.flatten()

image_1[:] = img1
image_2[:] = img2
image_1.flush()
image_2.flush()

In [4]:
def fpga_benchmark(mi_ip, fixed_buf, moving_buf, res_buf):
    mi_ip.write(0x10, fixed_buf.physical_address)
    mi_ip.write(0x18, moving_buf.physical_address)
    mi_ip.write(0x20, res_buf.physical_address)
    mi_ip.write(0x00, 1)
    while mi_ip.read(0x00) & 0x04 != 0x04:
        pass
    
def fpga_benchmark_2(mi_ip, fixed_buf, moving_buf, res_buf):
    mi_ip.write(0x10, fixed_buf.physical_address)
    mi_ip.write(0x18, moving_buf.physical_address)
    mi_ip.write(0x20, fixed_buf.physical_address)
    mi_ip.write(0x28, moving_buf.physical_address)
    mi_ip.write(0x30, res_buf.physical_address)
    mi_ip.write(0x00, 1)
    while mi_ip.read(0x00) & 0x04 != 0x04:
        pass

def cpu_benchmark(func, fixed, moving, res):
    func(fixed, moving, 512*512, res)

In [5]:
rails = pynq.get_rails()
recorder = pynq.DataRecorder(rails['12V'].power)

In [6]:
fpga_iter = 10_000
iter = 1_000

with recorder.record(0.1):
    time.sleep(5)
    
    recorder.mark()
    
    for i in range(10_000_000):
        pass
    
    recorder.mark()
    
    
    start = timer()
    for i in range(fpga_iter):
        fpga_benchmark(mi_ip, image_1, image_2, res)
    end = timer()
    fpga_time = (end-start)/fpga_iter
    print('fpga: {} (avg per iter: {})'.format(end-start, fpga_time))

    recorder.mark()
    
    start = timer()
    for i in range(iter):
        cpu_benchmark(_lib.parzen_mutual_information_matrix, img1, img2, res_cpu)
    end = timer()
    cpu_time = (end-start)/iter
    print('cpu: {} (avg per iter: {})'.format(end-start, cpu_time))
          
    print('speedup: {}x'.format(cpu_time/fpga_time))

fpga: 22.696682177000184 (avg per iter: 0.0022696682177000185)
cpu: 15.279358624999986 (avg per iter: 0.015279358624999986)
speedup: 6.73197893235841x


In [7]:
idle_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 0])
loop_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 1])
fpga_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 2])
cpu_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 3])

In [8]:
print(fpga_p - loop_p)
print(cpu_p - idle_p)
print('power gain: {}x'.format((fpga_p - loop_p)/(cpu_p - idle_p)))

0.2365953188054899
0.2690603864734289
power gain: 0.879339102669634x


In [9]:
overlay = Overlay('./build/assets/mutual_information/gem_wrapper.bit')
mi_ip = overlay.parzen_master_0

In [10]:
fpga_iter = 10_000
iter = 1_000

with recorder.record(0.1):
    start = timer()
    for i in range(fpga_iter):
        fpga_benchmark(mi_ip, image_1, image_2, res)
    end = timer()
    fpga_time = (end-start)/fpga_iter
    print('fpga: {} (avg per iter: {})'.format(end-start, fpga_time))

    recorder.mark()
    
    start = timer()
    for i in range(iter):
        cpu_benchmark(_lib.parzen_mutual_information_point, img1, img2, res_cpu)
    end = timer()
    cpu_time = (end-start)/iter
    print('cpu: {} (avg per iter: {})'.format(end-start, cpu_time))
    
    print('speedup: {}x'.format(cpu_time/fpga_time))

fpga: 7.920659188000172 (avg per iter: 7.920659188000173e-05)
cpu: 12.903212359999998 (avg per iter: 0.012903212359999998)
speedup: 162.9057891993183x


In [11]:
fpga_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 4])
cpu_p = np.mean(recorder.frame['12V_power'][recorder.frame.Mark == 5])

In [12]:
print(fpga_p - loop_p)
print(cpu_p - idle_p)
print('power gain: {}x'.format((fpga_p - loop_p)/(cpu_p - idle_p)))

0.10039757869249755
0.37406521739130305
power gain: 0.2683959214188937x
